# ClickHouse

## Import libraries and creating connection

In [1]:
#!pip install clickhouse_driver

In [2]:
from clickhouse_driver import Client


user_name = 'hardda_student_a.nikitin-8'
pwd = '0ddbfab04207297e7de7f96e697814e152de44e69f9fd4d8d5bdbb6b6107518a'

# creating connection ClickHouse
client = Client(host='clickhouse.lab.karpov.courses', port=9000,
                user=user_name, password=pwd, database='hardda')

# checking connection
result = client.execute("SELECT * FROM user_dm_events LIMIT 10")

# showing the result
for row in result[0:1]:
    print(row)

(datetime.date(2022, 2, 1), datetime.date(2022, 1, 31), 'android', 'f7411212fd0e2523e126cbfdd3f226c211212', '4beb10e1-aeeb-4c52-acd2-ce1ddbc1fc24b10e1', 22, 11, 3, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0)


## Tasks

### Task 1. 

What is the size of the table `user_dm_events`?

In [3]:
query = '''
SELECT
  database,
  table,
  formatReadableSize(SUM(bytes)) AS size
FROM 
  system.parts
WHERE
  database = 'hardda'
    AND table = 'user_dm_events'
GROUP BY
  database, table
'''

In [4]:
result = client.execute(query)

In [5]:
for row in result:
    print(row)

('hardda', 'user_dm_events', '4.72 GiB')


### Task 2. 

How many rows does the table have?

In [6]:
query = '''
SELECT
  COUNT(*)
FROM 
  user_dm_events
'''

In [7]:
result = client.execute(query)

In [8]:
print(result)

[(55916675,)]


### Task 3. 

On which set of fields is the table partitioned by?

In [9]:
query = '''
SELECT
  table,
  partition_key
FROM 
  system.tables
WHERE
  database = 'hardda'
    AND table = 'user_dm_events'
'''

In [10]:
result = client.execute(query)

In [11]:
print(result)

[('user_dm_events', 'event_date')]


### Task 4. 

What set of fields is the primary key (in ClickHouse terms and definitions)?

In [12]:
query = '''
SELECT
  table,
  primary_key
FROM 
  system.tables
WHERE
  database = 'hardda'
    AND table = 'user_dm_events'
'''

In [13]:
result = client.execute(query)

In [14]:
print(result)

[('user_dm_events', 'event_date')]


### Task 5. 

What set of fields is the order by key (in ClickHouse terms and definitions)?

In [15]:
query = '''
SELECT
  table,
  sorting_key
FROM 
  system.tables
WHERE
  database = 'hardda'
    AND table = 'user_dm_events'
'''

In [16]:
result = client.execute(query)

In [17]:
print(result)

[('user_dm_events', 'event_date')]


### Task 6.

What set of fields is the primary key (in terms of relational theory)?

In [18]:
query = '''
SELECT
  COUNT(*),
  COUNT(DISTINCT(user_pseudo_id || user_x_phone_id || toString(event_date)))
FROM 
  user_dm_events
'''

In [19]:
result = client.execute(query)

In [20]:
print(result)

[(55916675, 55916675)]


Answer: `user_pseudo_id` + `user_x_phone_id` + `event_date`

### Task 7. 

Which columns having a string data type can be transformed into LowCardinality data type to improve columns typization.

From official documentation: `use LowCardinality when you have up to 10,000 unique values of a column`.

In [21]:
query = '''
SELECT
  COUNT(DISTINCT(platform)),
  COUNT(DISTINCT(user_pseudo_id)),
  COUNT(DISTINCT(user_x_phone_id))
FROM 
  user_dm_events
'''

In [22]:
result = client.execute(query)

In [23]:
print(result)

[(2, 6663048, 6446931)]


Answer: `platform`

### Task 8. 

Choose numeric fields for which the data type can be changed to a more compact one.

**Int Ranges**  

**`Int8`** — [-128 : 127]  
**`Int16`** — [-32768 : 32767]  
**`Int32`** — [-2147483648 : 2147483647]  
**`Int64`** — [-9223372036854775808 : 9223372036854775807]  
**`Int128`** — [-170141183460469231731687303715884105728 : 170141183460469231731687303715884105727]  
**`Int256`** — [-57896044618658097711785492504343953926634992332820282019728792003956564819968 : 57896044618658097711785492504343953926634992332820282019728792003956564819967]

**UInt Ranges**

**`UInt8`** — [0 : 255]  
**`UInt16`** — [0 : 65535]  
**`UInt32`** — [0 : 4294967295]  
**`UInt64`** — [0 : 18446744073709551615]  
**`UInt128`** — [0 : 340282366920938463463374607431768211455]  
**`UInt256`** — [0 : 115792089237316195423570985008687907853269984665640564039457584007913129639935]

First let's check what data types do we have. 

In [24]:
query = '''
SELECT
  name,
  type
FROM 
  system.columns
WHERE
  database = 'hardda'
    AND table = 'user_dm_events'
'''

In [25]:
result = client.execute(query)

In [26]:
for i in result:
    print(i)

('event_date', 'Date')
('week_start_date', 'Date')
('platform', 'String')
('user_pseudo_id', 'String')
('user_x_phone_id', 'String')
('cnt_events', 'UInt32')
('cnt_view_advertisement', 'UInt32')
('cnt_view_listing', 'UInt32')
('cnt_new_advertisement_open', 'UInt32')
('cnt_new_advertisement_view_screen', 'UInt32')
('cnt_successful_new_advertisement_creation', 'UInt32')
('cnt_session_initiation', 'UInt32')
('cnt_display_phone', 'UInt32')
('cnt_send_message', 'UInt32')
('cnt_order_via_phone', 'UInt32')
('cnt_add_to_favorites', 'UInt32')
('cnt_view_ads_in_cabinet', 'UInt32')
('cnt_edit_advert_view_screen_package', 'UInt32')
('cnt_new_advert_view_screen_package', 'UInt32')


Now let's find out columns without an optimal data type. 

In [27]:
query = '''
SELECT
  max(cnt_events),
  max(cnt_view_advertisement),
  max(cnt_view_listing),
  max(cnt_new_advertisement_open),
  max(cnt_new_advertisement_view_screen),
  max(cnt_successful_new_advertisement_creation),
  max(cnt_session_initiation),
  max(cnt_display_phone),
  max(cnt_send_message),
  max(cnt_order_via_phone),
  max(cnt_add_to_favorites),
  max(cnt_view_ads_in_cabinet),
  max(cnt_edit_advert_view_screen_package),
  max(cnt_new_advert_view_screen_package)
FROM 
  user_dm_events
'''

In [28]:
result = client.execute(query)

In [29]:
print(result)

[(11070, 2902, 525, 346, 1713, 344, 49, 1588, 2683, 12, 1433, 960, 340, 688)]


Answer: all of the columns could be stored using a more compact `UInt16` data type. 

### Task 9. 

Choose fields with a date or date and time, for which the type can be changed to a more compact one.

ClickHouse supports the following date and time types:
- `Date` - A date without time. Stored as a 2-byte integer representing the number of days since 1970-01-01.
- `DateTime` - A date and time. Stored as a 8-byte integer representing the number of seconds since 1970-01-01 00:00:00 UTC.
- `DateTime64` - A high-precision date and time. Stored as a 16-byte integer representing the number of nanoseconds since 1970-01-01 00:00:00 UTC.
- `Date32` - A date without time. Stored as a 4-byte integer representing the number of days since 1970-01-01.

Let's check our date (datetime) columns data types. 

In [30]:
query = '''
SELECT
  name,
  type
FROM 
  system.columns
WHERE
  database = 'hardda'
    AND table = 'user_dm_events'
      AND (name = 'event_date' OR name = 'week_start_date')
'''

In [31]:
result = client.execute(query)

In [32]:
print(result)

[('event_date', 'Date'), ('week_start_date', 'Date')]


Answer: our date columns already have the most compact date type. No changes of data types required here. 

### Task 10. 

Optimize data types and save the data in a new table. What will be the size of a new table after data types optimization?

At first, we need to establish our connection to another database, where we can create new tables. 

In [33]:
client = Client(host='clickhouse.lab.karpov.courses', port=9000,
                user=user_name, password=pwd, database='hardda_student_data')

Then we need to create a new table with an updated data types. 

In [34]:
query = '''
DROP TABLE IF EXISTS dp_nav2103_user_dm_events_step1
'''

In [35]:
result = client.execute(query)

In [36]:
query = '''
CREATE TABLE dp_nav2103_user_dm_events_step1 (
  event_date Date,
  week_start_date Date,
  platform String,
  user_pseudo_id String,
  user_x_phone_id String,
  cnt_events UInt16,
  cnt_view_advertisement UInt16,
  cnt_view_listing UInt16,
  cnt_new_advertisement_open UInt16,
  cnt_new_advertisement_view_screen UInt16,
  cnt_successful_new_advertisement_creation UInt16,
  cnt_session_initiation UInt8,
  cnt_display_phone UInt16,
  cnt_send_message UInt16,
  cnt_order_via_phone UInt8,
  cnt_add_to_favorites UInt16,
  cnt_view_ads_in_cabinet UInt16,
  cnt_edit_advert_view_screen_package UInt16,
  cnt_new_advert_view_screen_package UInt16
) ENGINE = MergeTree() 
PARTITION BY event_date
ORDER BY event_date
SETTINGS index_granularity = 8192
'''

In [37]:
result = client.execute(query)

Then let's copy data from an old table to the new table. 

In [38]:
query = '''
INSERT INTO hardda_student_data.dp_nav2103_user_dm_events_step1
SELECT
  `event_date`,
  `week_start_date`,
  `platform`,
  `user_pseudo_id`,
  `user_x_phone_id`,
  CAST(`cnt_events` AS UInt16) AS `cnt_events`,
  CAST(`cnt_view_advertisement` AS UInt16) AS `cnt_view_advertisement`,
  CAST(`cnt_view_listing` AS UInt16) AS `cnt_view_listing`,
  CAST(`cnt_new_advertisement_open` AS UInt16) AS `cnt_new_advertisement_open`,
  CAST(`cnt_new_advertisement_view_screen` AS UInt16) AS `cnt_new_advertisement_view_screen`,
  CAST(`cnt_successful_new_advertisement_creation` AS UInt16) AS `cnt_successful_new_advertisement_creation`,
  CAST(`cnt_session_initiation` AS UInt8) AS `cnt_session_initiation`,
  CAST(`cnt_display_phone` AS UInt16) AS `cnt_display_phone`,
  CAST(`cnt_send_message` AS UInt16) AS `cnt_send_message`,
  CAST(`cnt_order_via_phone` AS UInt8) AS `cnt_order_via_phone`,
  CAST(`cnt_add_to_favorites` AS UInt16) AS `cnt_add_to_favorites`,
  CAST(`cnt_view_ads_in_cabinet` AS UInt16) AS `cnt_view_ads_in_cabinet`,
  CAST(`cnt_edit_advert_view_screen_package` AS UInt16) AS `cnt_edit_advert_view_screen_package`,
  CAST(`cnt_new_advert_view_screen_package` AS UInt16) AS `cnt_new_advert_view_screen_package`
FROM 
  hardda.user_dm_events
'''

In [39]:
result = client.execute(query)

Finally, let's check the size of a new table with an optimized data types. 

In [40]:
query = '''
SELECT
  database,
  table,
  formatReadableSize(SUM(bytes)) AS size
FROM 
  system.parts
WHERE
  database = 'hardda_student_data'
    AND table = 'dp_nav2103_user_dm_events_step1'
GROUP BY
  database, table
'''

In [41]:
result = client.execute(query)

In [42]:
print(result)

[('hardda_student_data', 'dp_nav2103_user_dm_events_step1', '4.45 GiB')]


### Task 11. 

How many rows does the new have?

In [43]:
query = '''
SELECT
  COUNT(*)
FROM 
  dp_nav2103_user_dm_events_step1
'''

In [44]:
result = client.execute(query)

In [45]:
print(result)

[(55916675,)]


### Task 12. 

What metric fields do not change their values from row to row in more than 90% of cases?

In [46]:
query = '''
with deltas AS (
SELECT
  neighbor(cnt_events, 1) - cnt_events AS delta_0,
  neighbor(cnt_view_advertisement, 1) - cnt_view_advertisement AS delta_1,
  neighbor(cnt_view_listing, 1) - cnt_view_listing AS delta_2,
  neighbor(cnt_new_advertisement_open, 1) - cnt_new_advertisement_open AS delta_3,
  neighbor(cnt_new_advertisement_view_screen, 1) - cnt_new_advertisement_view_screen AS delta_4,
  neighbor(cnt_successful_new_advertisement_creation, 1) - cnt_successful_new_advertisement_creation AS delta_5,
  neighbor(cnt_session_initiation, 1) - cnt_session_initiation AS delta_6,
  neighbor(cnt_display_phone, 1) - cnt_display_phone AS delta_7,
  neighbor(cnt_send_message, 1) - cnt_send_message AS delta_8,
  neighbor(cnt_order_via_phone, 1) - cnt_order_via_phone AS delta_9,
  neighbor(cnt_add_to_favorites, 1) - cnt_add_to_favorites AS delta_10,
  neighbor(cnt_view_ads_in_cabinet, 1) - cnt_view_ads_in_cabinet AS delta_11,
  neighbor(cnt_edit_advert_view_screen_package, 1) - cnt_edit_advert_view_screen_package AS delta_12,
  neighbor(cnt_new_advert_view_screen_package, 1) - cnt_new_advert_view_screen_package AS delta_13
FROM 
  dp_nav2103_user_dm_events_step1
ORDER BY event_date
)

SELECT
  quantile(0.91)(delta_0),
  quantile(0.91)(delta_1),
  quantile(0.91)(delta_2),
  quantile(0.91)(delta_3),
  quantile(0.91)(delta_4),
  quantile(0.91)(delta_5),
  quantile(0.91)(delta_6),
  quantile(0.91)(delta_7),
  quantile(0.91)(delta_8),
  quantile(0.91)(delta_9),
  quantile(0.91)(delta_10),
  quantile(0.91)(delta_11),
  quantile(0.91)(delta_12),
  quantile(0.91)(delta_13)
FROM deltas
'''

In [47]:
result = client.execute(query)

In [48]:
print(result)

[(86.0, 32.0, 5.0, 0.0, 0.0, 0.0, 3.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0)]


A list of metric fields do not change their values from row to row in more than 90% of cases:
- cnt_new_advertisement_open  
- cnt_new_advertisement_view_screen
- cnt_successful_new_advertisement_creation
- cnt_send_message
- cnt_order_via_phone
- cnt_edit_advert_view_screen_package
- cnt_new_advert_view_screen_package

### Task 13.

Apply compression codecs to the selected column\columns, creating a new table.

Creating a new table with data compression added.

In [49]:
query = '''
DROP TABLE IF EXISTS dp_nav2103_user_dm_events_step2
'''

In [50]:
result = client.execute(query)

In [51]:
query = '''
CREATE TABLE dp_nav2103_user_dm_events_step2 (
  event_date Date codec(DoubleDelta, LZ4),
  week_start_date Date codec(DoubleDelta, LZ4),
  platform String,
  user_pseudo_id String,
  user_x_phone_id String,
  cnt_events UInt16 codec(T64, LZ4),
  cnt_view_advertisement UInt16 codec(T64, LZ4),
  cnt_view_listing UInt16 codec(T64, LZ4),
  cnt_new_advertisement_open UInt16 codec(Delta, LZ4),
  cnt_new_advertisement_view_screen UInt16 codec(Delta, LZ4),
  cnt_successful_new_advertisement_creation UInt16 codec(Delta, LZ4),
  cnt_session_initiation UInt8 codec(T64, LZ4),
  cnt_display_phone UInt16 codec(T64, LZ4),
  cnt_send_message UInt16 codec(T64, LZ4),
  cnt_order_via_phone UInt8 codec(Delta, LZ4),
  cnt_add_to_favorites UInt16 codec(T64, LZ4),
  cnt_view_ads_in_cabinet UInt16 codec(T64, LZ4),
  cnt_edit_advert_view_screen_package UInt16 codec(Delta, LZ4),
  cnt_new_advert_view_screen_package UInt16 codec(Delta, LZ4)
) ENGINE = MergeTree()
PARTITION BY event_date
ORDER BY (user_pseudo_id, event_date)
SETTINGS index_granularity = 8192
'''

In [52]:
result = client.execute(query)

Copying data from an old table to the new table.

In [53]:
query = '''
INSERT INTO hardda_student_data.dp_nav2103_user_dm_events_step2
SELECT
  `event_date`,
  `week_start_date`,
  `platform`,
  `user_pseudo_id`,
  `user_x_phone_id`,
  CAST(`cnt_events` AS UInt16) AS `cnt_events`,
  CAST(`cnt_view_advertisement` AS UInt16) AS `cnt_view_advertisement`,
  CAST(`cnt_view_listing` AS UInt16) AS `cnt_view_listing`,
  CAST(`cnt_new_advertisement_open` AS UInt16) AS `cnt_new_advertisement_open`,
  CAST(`cnt_new_advertisement_view_screen` AS UInt16) AS `cnt_new_advertisement_view_screen`,
  CAST(`cnt_successful_new_advertisement_creation` AS UInt16) AS `cnt_successful_new_advertisement_creation`,
  CAST(`cnt_session_initiation` AS UInt8) AS `cnt_session_initiation`,
  CAST(`cnt_display_phone` AS UInt16) AS `cnt_display_phone`,
  CAST(`cnt_send_message` AS UInt16) AS `cnt_send_message`,
  CAST(`cnt_order_via_phone` AS UInt8) AS `cnt_order_via_phone`,
  CAST(`cnt_add_to_favorites` AS UInt16) AS `cnt_add_to_favorites`,
  CAST(`cnt_view_ads_in_cabinet` AS UInt16) AS `cnt_view_ads_in_cabinet`,
  CAST(`cnt_edit_advert_view_screen_package` AS UInt16) AS `cnt_edit_advert_view_screen_package`,
  CAST(`cnt_new_advert_view_screen_package` AS UInt16) AS `cnt_new_advert_view_screen_package`
FROM 
  hardda.user_dm_events
'''

In [54]:
result = client.execute(query)

 Checking the size of a new table with an optimized data types.

In [55]:
query = '''
SELECT
  database,
  table,
  formatReadableSize(SUM(bytes)) AS size
FROM 
  system.parts
WHERE
  database = 'hardda_student_data'
    AND table = 'dp_nav2103_user_dm_events_step2'
GROUP BY
  database, table
'''

In [56]:
result = client.execute(query)

In [57]:
print(result)

[('hardda_student_data', 'dp_nav2103_user_dm_events_step2', '4.21 GiB')]


### Task 14. 

How many rows does the new have?

In [58]:
query = '''
SELECT
  COUNT(*)
FROM 
  dp_nav2103_user_dm_events_step2
'''

In [59]:
result = client.execute(query)

In [60]:
print(result)

[(55916675,)]


### Task 15. 

Let's look at the contents of the `user_pseudo_id` and `user_x_phone_id` fields. Which of the fields can be converted to the UUID type?

In [61]:
query = '''
SELECT
  length(user_pseudo_id),
  length(user_x_phone_id)
FROM 
  dp_nav2103_user_dm_events_step2
LIMIT 5
'''

In [62]:
result = client.execute(query)

In [63]:
print(result)

[(37, 41), (37, 41), (37, 41), (37, 41), (37, 41)]


Answer: UUID is a 36-character string. Neither `user_pseudo_id` nor `user_x_phone_id` columns values are 36 long strings.

### Task 16.

Which data column can be calculated based on the values of another data column(s)?

In [64]:
query = '''
SELECT
  event_date,
  week_start_date,
  toStartOfWeek(event_date, 1) AS week_start_date_calc
FROM 
  dp_nav2103_user_dm_events_step2
LIMIT 1
'''

In [65]:
result = client.execute(query)

In [66]:
print(result)

[(datetime.date(2022, 2, 1), datetime.date(2022, 1, 31), datetime.date(2022, 1, 31))]


Answer: `week_start_date` can by derived from `event_date` with help of `toStartOfWeek` function. 

### Task 17. 

Apply compression codecs to the selected field(s), creating a new table with sorting by the fields `user_pseudo_id` and `event_date`, and partitioning by `year`.  
To this table, apply conversion of `user_pseudo_id` and `user_x_phone_id` to a Uint256 field with LZ4 compression  

How many rows are in the new table?

Creating a new table with new types of data compression added.

In [67]:
query = '''
DROP TABLE IF EXISTS dp_nav2103_user_dm_events_step3
'''

In [68]:
result = client.execute(query)

In [69]:
query = '''
CREATE TABLE dp_nav2103_user_dm_events_step3 (
  event_date Date codec(DoubleDelta, LZ4),
  week_start_date Date codec(DoubleDelta, LZ4),
  platform String,
  user_pseudo_id String,
  user_x_phone_id String, 
  cnt_events UInt16 codec(T64, LZ4),
  cnt_view_advertisement UInt16 codec(T64, LZ4),
  cnt_view_listing UInt16 codec(T64, LZ4),
  cnt_new_advertisement_open UInt16 codec(Delta, LZ4),
  cnt_new_advertisement_view_screen UInt16 codec(Delta, LZ4),
  cnt_successful_new_advertisement_creation UInt16 codec(Delta, LZ4),
  cnt_session_initiation UInt8 codec(T64, LZ4),
  cnt_display_phone UInt16 codec(T64, LZ4),
  cnt_send_message UInt16 codec(T64, LZ4),
  cnt_order_via_phone UInt8 codec(Delta, LZ4),
  cnt_add_to_favorites UInt16 codec(T64, LZ4),
  cnt_view_ads_in_cabinet UInt16 codec(T64, LZ4),
  cnt_edit_advert_view_screen_package UInt16 codec(Delta, LZ4),
  cnt_new_advert_view_screen_package UInt16 codec(Delta, LZ4),
  user_pseudo_id_comp ALIAS reinterpretAsUInt256(unhex(user_pseudo_id)),
  user_x_phone_id_comp ALIAS reinterpretAsUInt256(unhex(user_x_phone_id))
) ENGINE = MergeTree()
PARTITION BY toYear(event_date)
ORDER BY (user_pseudo_id, event_date)
SETTINGS index_granularity = 8192
'''

In [70]:
result = client.execute(query)

Copying data from an old table to the new table.

In [71]:
query = '''
INSERT INTO hardda_student_data.dp_nav2103_user_dm_events_step3
SELECT
  `event_date`,
  `week_start_date`,
  `platform`,
  `user_pseudo_id` AS user_pseudo_id_comp,
  `user_x_phone_id` AS user_x_phone_id_comp,
  CAST(`cnt_events` AS UInt16) AS `cnt_events`,
  CAST(`cnt_view_advertisement` AS UInt16) AS `cnt_view_advertisement`,
  CAST(`cnt_view_listing` AS UInt16) AS `cnt_view_listing`,
  CAST(`cnt_new_advertisement_open` AS UInt16) AS `cnt_new_advertisement_open`,
  CAST(`cnt_new_advertisement_view_screen` AS UInt16) AS `cnt_new_advertisement_view_screen`,
  CAST(`cnt_successful_new_advertisement_creation` AS UInt16) AS `cnt_successful_new_advertisement_creation`,
  CAST(`cnt_session_initiation` AS UInt8) AS `cnt_session_initiation`,
  CAST(`cnt_display_phone` AS UInt16) AS `cnt_display_phone`,
  CAST(`cnt_send_message` AS UInt16) AS `cnt_send_message`,
  CAST(`cnt_order_via_phone` AS UInt8) AS `cnt_order_via_phone`,
  CAST(`cnt_add_to_favorites` AS UInt16) AS `cnt_add_to_favorites`,
  CAST(`cnt_view_ads_in_cabinet` AS UInt16) AS `cnt_view_ads_in_cabinet`,
  CAST(`cnt_edit_advert_view_screen_package` AS UInt16) AS `cnt_edit_advert_view_screen_package`,
  CAST(`cnt_new_advert_view_screen_package` AS UInt16) AS `cnt_new_advert_view_screen_package`
FROM 
  hardda.user_dm_events
'''

In [72]:
result = client.execute(query)

Checking the number of rows we have in the new table. 

In [73]:
query = '''
SELECT
  COUNT(*)
FROM 
  dp_nav2103_user_dm_events_step3
'''

In [74]:
result = client.execute(query)

In [75]:
print(result)

[(55916675,)]


### Task 18. 

What is the size of the new table?

In [76]:
query = '''
SELECT
  database,
  table,
  formatReadableSize(SUM(bytes)) AS size
FROM 
  system.parts
WHERE
  database = 'hardda_student_data'
    AND table = 'dp_nav2103_user_dm_events_step3'
GROUP BY
  database, table
'''

In [77]:
result = client.execute(query)

In [78]:
print(result)

[('hardda_student_data', 'dp_nav2103_user_dm_events_step3', '5.41 GiB')]


### Task 19. 

Optimeze table `live_adverts`

Let's check the table first. 

In [79]:
query = '''
SELECT
  *
FROM 
  hardda.live_adverts
LIMIT 1
'''

In [80]:
result = client.execute(query)

In [81]:
print(result)

[(datetime.date(2022, 2, 1), 130486371, datetime.datetime(2012, 11, 6, 7, 12), 0, 'Москва', 123578197, 'unknown', 'Unknown', 'Unknown model', 123599132, '0', 0)]


How many rows does the table have?

In [82]:
query = '''
SELECT
  COUNT(*)
FROM 
  hardda.live_adverts
'''

In [83]:
result = client.execute(query)

In [84]:
print(result)

[(93120947,)]


What data types does it have?

In [85]:
query = '''
SELECT
  name,
  type
FROM 
  system.columns
WHERE
  database = 'hardda'
    AND table = 'live_adverts'
'''

In [86]:
result = client.execute(query)

In [87]:
for i in result:
    print(i)

('execution_date', 'Date32')
('advert_id', 'UInt64')
('created_at', 'DateTime')
('price', 'UInt64')
('region', 'LowCardinality(String)')
('user_id', 'UInt32')
('platform', 'LowCardinality(String)')
('auto_brand', 'LowCardinality(String)')
('auto_model', 'LowCardinality(String)')
('passport_id', 'UInt32')
('year', 'LowCardinality(String)')
('userType', 'UInt8')


Let's check maximum values in columns with integer data type. 

In [88]:
query = '''
SELECT
  max(advert_id),
  max(price),
  max(user_id),
  max(passport_id),
  max(userType)
FROM 
  hardda.live_adverts
'''

In [89]:
result = client.execute(query)

In [90]:
print(result)

[(271644234, 154014141415, 152308061, 144990126, 9)]


Checking size of the table. 

In [91]:
query = '''
SELECT
  database,
  table,
  formatReadableSize(SUM(bytes)) AS size
FROM 
  system.parts
WHERE
  database = 'hardda'
    AND table = 'live_adverts'
GROUP BY
  database, table
'''

In [92]:
result = client.execute(query)

In [93]:
print(result)

[('hardda', 'live_adverts', '1.71 GiB')]


Which set of fields is the table partitioned by?

In [94]:
query = '''
SELECT
  partition_key
FROM 
  system.tables
WHERE
  database = 'hardda'
    AND table = 'live_adverts'
'''

In [95]:
result = client.execute(query)

In [96]:
print(result)

[('DATE(execution_date)',)]


What set of fields is the primary key (in ClickHouse terms and definitions)?

In [97]:
query = '''
SELECT
  primary_key
FROM 
  system.tables
WHERE
  database = 'hardda'
    AND table = 'live_adverts'
'''

In [98]:
result = client.execute(query)

In [99]:
print(result)

[('execution_date, created_at',)]


What set of fields is the order by key (in ClickHouse terms and definitions)?

In [100]:
query = '''
SELECT
  sorting_key
FROM 
  system.tables
WHERE
  database = 'hardda'
    AND table = 'live_adverts'
'''

In [101]:
result = client.execute(query)

In [102]:
print(result)

[('execution_date, created_at',)]


Let's create an optimized table. 

In [103]:
query = '''
DROP TABLE IF EXISTS dp_live_adverts_nav2103
'''

In [104]:
result = client.execute(query)

In [105]:
query = '''
CREATE TABLE dp_live_adverts_nav2103 (
`execution_date` Date32 CODEC(DoubleDelta), 
`advert_id` UInt64 CODEC(T64), 
`created_at` DateTime CODEC(DoubleDelta), 
`price` UInt64 CODEC(Gorilla), 
`region` LowCardinality(String), 
`user_id` UInt32 CODEC(T64), 
`platform` LowCardinality(String), 
`auto_brand` LowCardinality(String), 
`auto_model` LowCardinality(String), 
`passport_id` UInt32 CODEC(T64), 
`year` LowCardinality(String), 
`userType` UInt8 CODEC(T64)
) ENGINE = MergeTree 
PARTITION BY DATE(execution_date) 
ORDER BY (execution_date, created_at, user_id, region) 
PRIMARY KEY (execution_date, created_at, user_id, region)
SETTINGS index_granularity = 8192
'''

In [106]:
result = client.execute(query)

Now we have to copy data in int. 

In [107]:
query = '''
INSERT INTO hardda_student_data.dp_live_adverts_nav2103
SELECT
  `execution_date`,
  `advert_id`,
  `created_at`,
  `price`,
  `region`,
  `user_id`,
  `platform`,
  `auto_brand`,
  `auto_model`,
  `passport_id`,
  `year`,
  `userType`
FROM 
  hardda.live_adverts
'''

In [108]:
result = client.execute(query)

Let's run some queries to test the optimized table. 

Test query 1. 

In [109]:
query = '''
WITH datas AS (
    SELECT  
        execution_date,
        region,
        CAST(advert_id AS Int64) AS adverts,
        CAST(passport_id AS Int64) AS listers
    FROM
        hardda_student_data.dp_live_adverts_nav2103
    WHERE execution_date between toDate32('2022-01-01') 
      and toDate32('2022-02-01')
),
pre AS (
    SELECT *
    FROM datas
    UNION ALL
    SELECT execution_date, region, adverts, listers
    FROM datas
)
SELECT
    execution_date,
    region,
    COUNT(DISTINCT adverts) AS unique_adverts,
    COUNT(DISTINCT listers) AS unique_listers,
    COUNT(DISTINCT adverts) / COUNT(DISTINCT listers) AS adverts_per_lister
FROM
    pre
GROUP BY  
    execution_date, region
LIMIT 5
'''

In [110]:
result = client.execute(query)

In [111]:
for i in result:
    print(i)

(datetime.date(2022, 2, 1), 'Уфа', 5412, 3743, 1.4458990114881112)
(datetime.date(2022, 2, 1), 'Пермь', 4350, 3251, 1.3380498308212858)
(datetime.date(2022, 2, 1), 'Екатеринбург', 14445, 8462, 1.7070432521862444)
(datetime.date(2022, 2, 1), 'Нижний Новгород', 10150, 7510, 1.351531291611185)
(datetime.date(2022, 2, 1), 'Омск', 6277, 4110, 1.5272506082725061)


Test query 2. 

In [112]:
query = '''
WITH data AS(
SELECT
  region,
  auto_brand,
  COUNT(*) AS count_lives
FROM
  hardda_student_data.dp_live_adverts_nav2103
WHERE
  auto_brand != 'Unknown'
GROUP BY
  region, auto_brand
),
pre AS(
  SELECT
    *,
    row_number() OVER (partition by region order by count_lives desc)  AS rn
  FROM
    data
)
SELECT region, auto_brand FROM pre
WHERE rn = 1
LIMIT 5
'''

In [113]:
result = client.execute(query)

In [114]:
for i in result:
    print(i)

('Волгоград', 'ВАЗ (Lada)')
('Воронеж', 'Toyota')
('Екатеринбург', 'ВАЗ (Lada)')
('Ижевск', 'ВАЗ (Lada)')
('Казань', 'ВАЗ (Lada)')


Test query 3. 

In [115]:
query = '''
SELECT
  user_id,
  COUNT(DISTINCT(advert_id)) AS advert_count
FROM
  hardda_student_data.dp_live_adverts_nav2103
GROUP BY
  user_id
ORDER BY
  advert_count DESC
LIMIT 5
'''

In [116]:
result = client.execute(query)

In [117]:
for i in result:
    print(i)

(126942552, 15553)
(123466420, 15512)
(125442172, 12995)
(123910335, 8977)
(144368376, 7251)


Everything works smooth and fine now!